In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn import svm

import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

from sklearn import neighbors
from math import sqrt
from sklearn.metrics import r2_score,mean_squared_error

#import the metrics 
from sklearn import metrics

from sklearn.preprocessing import scale 
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA



The data set comprises several sensor data collected from a permanent magnet synchronous motor (PMSM) deployed on a test bench. The PMSM represents a german OEM's prototype model. Test bench measurements were collected by the LEA department at Paderborn University.

# Content
All recordings are sampled at 2 Hz. The data set consists of multiple measurement sessions, which can be distinguished from each other by column "profile_id". A measurement session can be between one and six hours long.

The motor is excited by hand-designed driving cycles denoting a reference motor speed and a reference torque.
Currents in d/q-coordinates (columns "id" and iq") and voltages in d/q-coordinates (columns "ud" and "uq") are a result of a standard control strategy trying to follow the reference speed and torque.
Columns "motor_speed" and "torque" are the resulting quantities achieved by that strategy, derived from set currents and voltages.

Most driving cycles denote random walks in the speed-torque-plane in order to imitate real world driving cycles to a more accurate degree than constant excitations and ramp-ups and -downs would.

In [3]:
df = pd.read_csv('/kaggle/input/electric-motor-temperature/measures_v2.csv')

In [4]:
df.shape

In [5]:
df.head()

In [6]:
df.info()

In [7]:
df.describe()

In [8]:
df.isnull().sum()

In [9]:
df.dtypes.value_counts()

In [10]:
import seaborn as sns
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True)

In [11]:
for i in df.columns:
    plt.title(f'Distribution of feature {i}')
    plt.style.use('fivethirtyeight')
    df[i].plot(kind='hist')
    plt.figure(figsize=(12, 8))

In [12]:
import warnings
warnings.filterwarnings('ignore')
plt.figure(figsize=(12, 8))
sns.scatterplot(df['ambient'],df['pm'])

In [13]:
plt.figure(figsize=(12, 8))
sns.scatterplot(df['coolant'],df['pm'])


In [14]:
plt.figure(figsize=(12, 8))
sns.scatterplot(df['u_q'],df['pm'])

In [15]:
plt.figure(figsize=(12, 8))
sns.scatterplot(df['u_d'],df['pm'])


In [16]:
plt.figure(figsize=(12, 8))
sns.scatterplot(df['i_q'],df['pm'])


In [17]:
plt.figure(figsize=(12, 8))
sns.scatterplot(df['i_d'],df['pm'])


In [18]:
plt.figure(figsize=(12, 8))
sns.scatterplot(df['motor_speed'],df['pm'])


In [19]:
#not important so we will drop it from the data frame
df.drop('profile_id', axis=1, inplace=True )

In [20]:
x  = df.drop(['motor_speed'], axis=1)
y = df['motor_speed']

In [21]:
df.columns

In [22]:
x

In [23]:
from sklearn.model_selection import train_test_split, cross_val_score

x_train, x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)

training=df.copy()

In [24]:
x_train

In [25]:
x_test

In [26]:
x_train_scaled = scale(x_train)
x_test_scaled = scale(x_test)

In [27]:
# Define cross-validation folds
cv = KFold(n_splits=10, shuffle=True, random_state=42)
lin_reg = LinearRegression().fit(x_train_scaled, y_train)

# Get R2 score
r2 = lin_reg.score(x_train_scaled, y_train)

lr_score = -1 * cross_val_score(lin_reg, 
                                 x_train_scaled, 
                                 y_train, 
                                 cv=cv, 
                                 scoring='neg_root_mean_squared_error')
print(lr_score)

In [28]:
lr_score_train = np.mean(lr_score)
lr_score_train

In [29]:
#r2_score
r2

In [30]:
#predict
y_predicted = lin_reg.predict(x_test_scaled)
lr_score_test = mean_squared_error(y_test, y_predicted, squared=False)
lr_score_test

In [31]:
#Lasso Regression (L1 regularization)
lasso_reg = LassoCV().fit(x_train_scaled, y_train)


# Get R2 score
lasso_reg.score(x_train_scaled, y_train)

lasso_scores = -1 * cross_val_score(lasso_reg, 
                                    x_train_scaled, 
                                    y_train, 
                                    cv=cv, 
                                    scoring='neg_root_mean_squared_error')
print(lasso_scores)

In [32]:
lasso_score_train = np.mean(lasso_scores)
lasso_score_train

# Predict

In [33]:
y_predicted = lasso_reg.predict(x_test_scaled)
lasso_score_test = mean_squared_error(y_test, y_predicted, squared=False)
lasso_score_test

# Ridge Regression (L2 regularization)

In [34]:
ridge_reg = RidgeCV().fit(x_train_scaled, y_train)

In [35]:
# Get R2 score
ridge_reg.score(x_train_scaled, y_train)

In [36]:
ridge_scores = -1 * cross_val_score(ridge_reg, 
                                    x_train_scaled, 
                                    y_train, 
                                    cv=cv, 
                                    scoring='neg_root_mean_squared_error')
print(ridge_scores)

In [37]:
ridge_score_train = np.mean(ridge_scores)
print(ridge_score_train)

# Predict

In [38]:

y_predicted = ridge_reg.predict(x_test_scaled)
ridge_score_test = mean_squared_error(y_test, y_predicted, squared=False)
ridge_score_test

In [39]:
#Visualizing for Multiple Linear Regression y values

import seaborn as sns
ax1 = sns.distplot(y_train, hist=False, color="r", label="Actual Value")
sns.distplot(y_test, hist=False, color="b", label="Fitted Values" , ax=ax1);

In [40]:
pca = PCA()
x_train_pc = pca.fit_transform(x_train_scaled)

pd.DataFrame(pca.components_.T).loc[:10, :]

In [41]:
pca.explained_variance_ratio_

In [42]:
# Linear Regression
lin_reg = LinearRegression().fit(x_train_scaled, y_train)
lr_score_train = -1 * cross_val_score(lin_reg, x_train_scaled, y_train, cv=cv, scoring='neg_root_mean_squared_error').mean()
lr_score_test = mean_squared_error(y_test, lin_reg.predict(x_test_scaled), squared=False)

In [43]:
# Initialize linear regression instance
lin_reg2 = LinearRegression()

# Create empty list to store RMSE for each iteration
rmse_list = []

# Loop through different count of principal components for linear regression
for i in range(1, x_train_pc.shape[1]+1):
    rmse_score = -1 * cross_val_score(lin_reg2, 
                                      x_train_pc[:,:i], # Use first k principal components
                                      y_train, 
                                      cv=cv, 
                                      scoring='neg_root_mean_squared_error').mean()
    rmse_list.append(rmse_score)
    
# Visual analysis - plot RMSE vs count of principal components used
plt.plot(rmse_list, '-o')
plt.xlabel('Number of principal components in regression')
plt.ylabel('RMSE')
plt.title('Quality')
plt.xlim(xmin=-1);
plt.xticks(np.arange(x_train_pc.shape[1]), np.arange(1, x_train_pc.shape[1]+1))
plt.axhline(y=lr_score_train, color='g', linestyle='-');

In [44]:
# Visually determine optimal number of principal components
best_pc_num = 8

# Train model with first 8 principal components
lin_reg_pc = LinearRegression().fit(x_train_pc[:,:best_pc_num], y_train)

# Get cross-validation RMSE (train set)
pcr_score_train = -1 * cross_val_score(lin_reg_pc, 
                                       x_train_pc[:,:best_pc_num], 
                                       y_train, 
                                       cv=cv, 
                                       scoring='neg_root_mean_squared_error').mean()

# Train model on training set
lin_reg_pc = LinearRegression().fit(x_train_pc[:,:best_pc_num], y_train)

# Get first 8 principal components of test set
x_test_pc = pca.transform(x_test_scaled)[:,:best_pc_num]

# Predict on test data
preds = lin_reg_pc.predict(x_test_pc)
pcr_score_test = mean_squared_error(y_test, preds, squared=False)

In [45]:
pcr_score_test

In [46]:
train_metrics = np.array([round(lr_score_train,3), 
                          round(lasso_score_train,3), 
                          round(ridge_score_train,3), 
                          round(pcr_score_train,3)]) 
train_metrics = pd.DataFrame(train_metrics, columns=['RMSE (Train Set)'])
train_metrics.index = ['Linear Regression', 
                       'Lasso Regression', 
                       'Ridge Regression', 
                       f'PCR ({best_pc_num} components)']
train_metrics

In [47]:
test_metrics = np.array([round(lr_score_test,3), 
                         round(lasso_score_test,3), 
                         round(ridge_score_test,3), 
                         round(pcr_score_test,3)]) 
test_metrics = pd.DataFrame(test_metrics, columns=['RMSE (Test Set)'])
test_metrics.index = ['Linear Regression', 
                      'Lasso Regression', 
                      'Ridge Regression', 
                      f'PCR ({best_pc_num} components)']
test_metrics

In [48]:
#Visualizing for Multiple Linear Regression y values

import seaborn as sns
ax1 = sns.distplot(y_train, hist=False, color="r", label="Actual Value")
sns.distplot(y_test, hist=False, color="b", label="Fitted Values" , ax=ax1);

#lets try in different way

In [49]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=100)

In [50]:
## Scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [51]:
## Training the dataset
from sklearn.linear_model import  Ridge, Lasso, LinearRegression, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,  AdaBoostRegressor, BaggingRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [52]:
def predict(model):
    # Define Models Name
    print('model: {}'.format(model))
    ## fitting the model
    model.fit(x_train_scaled,y_train)
    ## predicting the value
    y_pred = model.predict(x_test_scaled)
    print('R2 score',r2_score(y_test,y_pred))
    print('MAE',mean_absolute_error(y_test,y_pred))
    print('RMSE:{}'.format(np.sqrt(mean_squared_error(y_test,y_pred))))
    plt.figure(figsize=(12, 8))
    sns.distplot(y_test-y_pred)

In [53]:
import seaborn as sns
plt.figure(figsize=(12, 8))

In [54]:
predict(LinearRegression())

In [55]:
predict(Ridge())

In [56]:
predict(Lasso())

In [57]:
predict(DecisionTreeRegressor())

In [58]:
predict(KNeighborsRegressor())

In [59]:
predict(XGBRegressor())

In [60]:
from catboost import CatBoostRegressor

cbr_model = CatBoostRegressor(n_estimators=200, 
                             loss_function='MAE',
                             eval_metric='RMSE')

cbr_model.fit(x_train, y_train)

y_predict = cbr_model.predict(x_test)
r2_score(y_test, y_predict)

In [62]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler

ct =  make_column_transformer((MinMaxScaler(),['u_q', 'coolant', 'stator_winding', 'u_d', 'stator_tooth', 'i_d', 'i_q',
       'pm', 'stator_yoke', 'ambient', 'torque']))

In [63]:
x = df.drop(['motor_speed'], axis=1)
y = df['motor_speed']

In [64]:
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
x_train , x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
                                                     random_state=42)

ct.fit(x_train)

x_train_normal = ct.transform(x_train)
x_test_normal = ct.transform(x_test)

In [65]:
import tensorflow as tf
tf.random.set_seed(42)



# Build the model (3 layers, 100, 10, 1 units)
model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(64),
   tf.keras.layers.Dense(32),
  tf.keras.layers.Dense(1)
])

# Compile the model
model_1.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mse'])

# Fit the model for 200 epochs (same as insurance_model_2)
history = model_1.fit(x_train_normal, y_train, epochs=10, verbose=100) 



In [66]:
model_1_loss, model_1_mae = model_1.evaluate(x_test_normal, y_test)

In [67]:

# Compare modelling results from non-normalized data and normalized data
model_1_loss , model_1_mae

In [68]:
# Check out our second model's architecture
model_1.summary()